### Vector Store and Retriever

In [2]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="Dogs are very friendly and make good pets",
        metadata={
            "source":"https://www.example.com/dog"
        }
    ),
    Document(
        page_content="Cats are very independent and make bad pets",
        metadata={
            "source":"https://www.example.com/cat"
        }
    ),
    Document(
        page_content="Rabbits are very friendly and make good pets",
        metadata={
            "source":"https://www.example.com/dog"
        }
    )

]

In [3]:
documents

[Document(metadata={'source': 'https://www.example.com/dog'}, page_content='Dogs are very friendly and make good pets'),
 Document(metadata={'source': 'https://www.example.com/cat'}, page_content='Cats are very independent and make bad pets'),
 Document(metadata={'source': 'https://www.example.com/dog'}, page_content='Rabbits are very friendly and make good pets')]

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001C8543E76A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C8543E6BC0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
## Embedding
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store=Chroma.from_documents(documents,embeddings)
vector_store

In [10]:
vector_store.similarity_search("dogs")




[Document(id='eccefebc-e3ec-4980-b453-6f984b305d81', metadata={'source': 'https://www.example.com/dog'}, page_content='Dogs are very friendly and make good pets'),
 Document(id='81ccde32-9509-4219-904c-5715bd9dcd67', metadata={'source': 'https://www.example.com/cat'}, page_content='Cats are very independent and make bad pets'),
 Document(id='8154e336-9d8b-4dd4-8b79-cc7bb8dea516', metadata={'source': 'https://www.example.com/dog'}, page_content='Rabbits are very friendly and make good pets')]

In [11]:
#Async query
await vector_store.asimilarity_search("dogs")

[Document(id='eccefebc-e3ec-4980-b453-6f984b305d81', metadata={'source': 'https://www.example.com/dog'}, page_content='Dogs are very friendly and make good pets'),
 Document(id='81ccde32-9509-4219-904c-5715bd9dcd67', metadata={'source': 'https://www.example.com/cat'}, page_content='Cats are very independent and make bad pets'),
 Document(id='8154e336-9d8b-4dd4-8b79-cc7bb8dea516', metadata={'source': 'https://www.example.com/dog'}, page_content='Rabbits are very friendly and make good pets')]

In [13]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriever=RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["dogs","cats"])


[[Document(id='eccefebc-e3ec-4980-b453-6f984b305d81', metadata={'source': 'https://www.example.com/dog'}, page_content='Dogs are very friendly and make good pets')],
 [Document(id='81ccde32-9509-4219-904c-5715bd9dcd67', metadata={'source': 'https://www.example.com/cat'}, page_content='Cats are very independent and make bad pets')]]

In [14]:
retriever=vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":1
    }
)
retriever.batch(["dogs", "cats"])

[[Document(id='eccefebc-e3ec-4980-b453-6f984b305d81', metadata={'source': 'https://www.example.com/dog'}, page_content='Dogs are very friendly and make good pets')],
 [Document(id='81ccde32-9509-4219-904c-5715bd9dcd67', metadata={'source': 'https://www.example.com/cat'}, page_content='Cats are very independent and make bad pets')]]

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer the question using the provided context only.
{question}

context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)
response=rag_chain.invoke("What are dogs?")
response.content

'Dogs are very friendly and make good pets.'